# Data preprocessing

In [4]:
import pandas as pd
# Import data
df_raw = pd.read_excel('../Data/DATA_UFM_combined.xlsx', header=0)

# Columns to keep
cols = [
    'udbud_id',
    'titel',
    'educational_category',
    'displaydocclass',
    'hovedinsttx',
    'instregiontx',
    'instkommunetx',
    'arbejdstid_timer',
    'arbmedstud_likert',
    'ensom_likert',
    'maanedloen_10aar',
    'maanedloen_nyudd',
    'tidsforbrug_p50',
]

data = df_raw[cols]

# Remove all udbud_id==999999, as this is the education on national level
data_whole_edu = data[data['udbud_id'] == 999999]
data = data[data['udbud_id'] != 999999]

# Remove the udbud_id column
data = data.drop(columns=['udbud_id'])

data_na = data.copy()
# Remove all rows with missing values
data = data.dropna()

## Input experiment

In [31]:
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, show
from bokeh.models import TextInput
from bokeh.layouts import layout


p = figure(x_range=data['educational_category'].unique().tolist(), height=400,
           tools="", toolbar_location=None, title="Counts of Educational Categories")
# Make a bar chart of educational_category counts

colors = p.vbar(x=data['educational_category'].unique().tolist(), top=data['educational_category'].value_counts(), width=0.5)
# p.vbar(x='educational_category', top=)

p.xgrid.grid_line_color = None
p.y_range.start = 0

textinput = TextInput(value=colors.glyph.fill_color, title="Label:")
textinput.js_link("value", colors.glyph, "fill_color")

layout = layout([textinput], [p])

show(layout)


## Brush and linking experiment

In [44]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import BoxSelectTool, BoxZoomTool, LassoSelectTool

x = list(range(21))
y0 = x
y1 = [20-xx for xx in x]
y2 = [abs(xx-10) for xx in x]

# create a new plot
s1 = figure(width=250, height=250, title=None)
s1.scatter(x, y0, size=10, color="navy", alpha=0.5)

# create a new plot and share both ranges
s2 = figure(width=250, height=250, x_range=s1.x_range, y_range=s1.y_range, title=None, tools=[BoxSelectTool(), LassoSelectTool(), BoxZoomTool()])
s2.scatter(x, y1, size=10, marker="triangle", color="firebrick", alpha=0.5)

# create a new plot and share only one range
s3 = figure(width=250, height=250, x_range=s1.x_range, title=None, tools=[BoxSelectTool(), LassoSelectTool(), BoxZoomTool()])
s3.scatter(x, y2, size=10, marker="square", color="olive", alpha=0.5)

p = gridplot([[s1, s2, s3]])#, toolbar_location=None)

p.toolbar.autohide = True


show(p)

In [38]:
import numpy as np

from bokeh.models import BoxSelectTool, BoxZoomTool, LassoSelectTool
from bokeh.plotting import figure, show

x = np.random.random(size=200)
y = np.random.random(size=200)

# Basic plot setup
plot = figure(width=400, height=400, title='Select and Zoom',
              tools="box_select,box_zoom,lasso_select,reset")

plot.scatter(x, y, size=5)

select_overlay = plot.select_one(BoxSelectTool).overlay

select_overlay.fill_color = "firebrick"
select_overlay.line_color = None

zoom_overlay = plot.select_one(BoxZoomTool).overlay

zoom_overlay.line_color = "olive"
zoom_overlay.line_width = 8
zoom_overlay.line_dash = "solid"
zoom_overlay.fill_color = None

plot.select_one(LassoSelectTool).overlay.line_dash = [10, 10]

show(plot)